PURPOSE OF CODE

Cleaning meaningless words and punctuations to prepare data to further train and test datasets seperation. Since test data should also be cleaned before scoring this process shifted to the beginning and done over whole data before test and train devision.

PROCESS NOTES

WILL BE DONE
 - ..\AppData\Roaming\nltk_data\corpora\stopwords\english_redesigned document could be checked for stopwords
 - numbers should be changed to their word forms

DONE
- all phrases are lowered
- stemmed using nltk PorterStemmer
- punctuations are erased
- stop words (nltk library pre created stop words list is redesigned to use) are erased
- dublicate phrases with different sentiment scores are averaged
- saved as clean-data.csv

Get data under folder /data

In [1]:
from IPython.core.debugger import Tracer; debug = Tracer()
import numpy as np
import pandas as pd
df=pd.read_csv('data/data.csv',sep="\t")
df[89:93]

,PhraseId,SentenceId,Phrase,Sentiment
89,90,3,Ismail,2
90,91,3,Merchant 's,2
91,92,3,Merchant,2
92,93,3,'s,2


Replace capitals to non-capitals.

In [2]:
df.Phrase=df.Phrase.str.lower()
df[89:93]

,PhraseId,SentenceId,Phrase,Sentiment
89,90,3,ismail,2
90,91,3,merchant 's,2
91,92,3,merchant,2
92,93,3,'s,2


There are no 'isn't', 'arent', 'havent' or such words in data, they all have already changed to their not seperated forms like "is not" . For example check for 'isn't'.

In [3]:
haha=0
for x in df.Phrase.str.contains(" isn't"):
    if x:
        haha+=1
haha

0

Stem the data using nltk PorterStemmer which changes the words like "pythoner", "pythoning" etc. to the "python".

In [4]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

def stemming(x):
    proper_words=[]
    words=x.split()
    for word in words:
        proper_words.append(ps.stem(word))
    return " ".join(proper_words)

df.Phrase=df.Phrase.map(stemming)

Remove punctuations.

In [5]:
df.Phrase = df.Phrase.str.replace('[^\w\s]','')
df[89:93]

,PhraseId,SentenceId,Phrase,Sentiment
89,90,3,ismail,2
90,91,3,merchant s,2
91,92,3,merchant,2
92,93,3,s,2


nltk's english stopwords list at location "..\AppData\Roaming\nltk_data\corpora\stopwords" includes words like "no", "not". They changes the meaning in sentiment analysis since if "no" is erased from data "not good" and "good" scored as same. So stopwords are removed by using redesigned form of pre-created list of nltk library.

The words erased from list are
"not, no, very, too, up, down, in, out, over, under, few"

Check again if miss.

In [6]:
#import redesigned stopword list named as "english_redesigned" from nltk
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english_redesigned'))

#define a map to delete stopwords under each phrase
def stop_erase(x):
    proper_words = []
    words = x.split()
    for word in words:
        if not word in stopwords_set:
           proper_words.append(word)
        
    return " ".join(proper_words)

df.Phrase=df.Phrase.map(stop_erase)
df[89:93]

,PhraseId,SentenceId,Phrase,Sentiment
89,90,3,ismail,2
90,91,3,merchant,2
91,92,3,merchant,2
92,93,3,,2


Replace empty phrase rows to NaN then delete.

In [7]:
print("size of data with empty phrase rows=",len(df.index))

#define a map to change empty phrases to NaN
def change_nan(x):
    if pd.isnull(x):
        return np.nan 
    if len(x) == 0:
        return np.nan
    return x

df['Phrase'] = df.Phrase.map(change_nan)

#remove NaN rows of df
df=df.dropna()

#reset indexing
df=df.reset_index(drop=True)

print("size of data after empty phrase rows deleted=",len(df.index))

size of data with empty phrase rows= 156060
size of data after empty phrase rows deleted= 155259


Afer clean, the same same phrases with differen sentiment scores created as follows.

In [8]:
df[14:16]

,PhraseId,SentenceId,Phrase,Sentiment
14,20,1,good goos,2
15,22,1,good goos,3


Take a phrase as referance to the "phrase" list. Find the same ones in data and record the indexes to the "same" list. In such same phrase groups assign the first one as referance by changing its sentiment score to group average and change other phrases to NaN to remove later from the list. Before doing that sentiment scores should be converted into float data type from intiger(as a default).

In [9]:
print("data type of sentiment scores= ",df.Sentiment.dtype)

#convert sentiment scores to float to write averaged ones
df.Sentiment=df.Sentiment.astype(float)
df.dtypes
print("data type of sentiment scores after conversion= ",df.Sentiment.dtype)

#write size of data before merge
print("data size before merge= ",len(df.index))

#"phrase" holds the current phrase to compare through whole set and find the indexes of same other phrase rows 
#which will written in "same"
#then they will changed to NaN for further clean process
#summation holds the summation of same phrase sentiment scores that will avaraged and written to the first phrase among all same phrases
same=[]
summation=0
phrase=[]

for index,row in df.iterrows():
    if row.Phrase!=np.nan :
        same=df.Phrase[df.Phrase==row.Phrase]
        if len(same.index)>1:
            summation=0
            phrase=df.loc[same.index[0],'Phrase']
            for x in range(1,len(same.index)):
                df.loc[same.index[x],'Phrase']=np.nan
                summation+=df.loc[same.index[x],'Sentiment']
            summation+=df.loc[same.index[0],'Sentiment']
            df.loc[same.index[0],'Sentiment']=summation/len(same.index)

#clean all NaN rows
df=df.dropna()

#write size of data after merge
print("data size after merge= ",len(df.index))

data type of sentiment scores=  int64
data type of sentiment scores after conversion=  float64
data size before merge=  155259
data size after merge=  85812


In [10]:
df[14:16]

,PhraseId,SentenceId,Phrase,Sentiment
24,32,1,good gander occasion amus none amount much stori,2.0
25,33,1,gander occasion amus none amount much stori,1.5


Remove NaN assigneds and reindex data.

In [11]:
df=df.dropna()
df=df.reset_index(drop=True)
df.head(15)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos also good...,1.000000
1,2,1,seri escapad demonstr adag good goos,2.000000
2,3,1,seri,2.000000
3,6,1,escapad demonstr adag good goos,2.000000
4,9,1,escapad,2.000000
5,10,1,demonstr adag good goos,2.000000
6,11,1,demonstr adag,2.000000
7,12,1,demonstr,2.000000
8,13,1,adag,2.000000
9,16,1,good goos,2.250000


This cleaned data is sort of vocabulary list. Save file as cleandf.csv under data folder

In [12]:
df.to_csv("data/clean-data.csv", sep='\t')
df.describe()

,PhraseId,SentenceId,Sentiment
count,85812.000000,85812.000000,85812.000000
mean,76983.701732,4023.147462,2.060142
std,45395.101649,2519.421455,0.869558
min,1.000000,1.000000,0.000000
25%,37399.750000,1776.000000,1.666667
50%,76569.000000,3938.000000,2.000000
75%,116410.250000,6207.000000,2.500000
max,156060.000000,8544.000000,4.000000
